In [1]:
import pandas as pd

Imagine we have the following responses in a survey

In [22]:
df = pd.DataFrame(data=
        [['slightly influenced', 'slightly influenced',
        'slightly influenced'],
       ['highly influenced', 'highly influenced', 'not at all'],
       ['influenced', 'highly influenced', 'not at all'],
       ['influenced', 'influenced', 'moderately influenced'],
       ['slightly influenced', 'slightly influenced',
        'slightly influenced'],
       ['moderately influenced', 'slightly influenced',
        'slightly influenced'],
       ['moderately influenced', 'influenced', 'moderately influenced'],
       ['influenced', 'highly influenced', 'slightly influenced'],
       ['influenced', 'influenced', 'slightly influenced'],
       ['highly influenced', 'slightly influenced',
        'slightly influenced']], columns=['factor_one', 'factor_two', 'factor_three'], )

In [23]:
df.index = ['person_%d'%i for i in range(10)]

### Connections between individuals
We will connect people if they are *both* `influenced` or `highly influenced` by at least one of the 3 factors. We first better convert `influenced` or `highly influenced` to `1` and the other values to `0`.

In [24]:
df_converted = df.applymap(lambda x: 
                             1 if x in ('influenced', 'high influenced') else 0)

Now make an _adjacency_ DataFrame by computing the inner product of `df_converted` with its transpopse `df_converted.T`. An effect of the inner product, is that tthe value element `(i,j)` of the matrix, will count the number of similar influences of respondent `i` with respondent `j`.

In [25]:
df_adj = df_converted @ df_converted.T

In [26]:
df_adj

,person_0,person_1,person_2,person_3,person_4,person_5,person_6,person_7,person_8,person_9
person_0,0,0,0,0,0,0,0,0,0,0
person_1,0,0,0,0,0,0,0,0,0,0
person_2,0,0,1,1,0,0,0,1,1,0
person_3,0,0,1,2,0,0,1,1,2,0
person_4,0,0,0,0,0,0,0,0,0,0
person_5,0,0,0,0,0,0,0,0,0,0
person_6,0,0,0,1,0,0,1,0,1,0
person_7,0,0,1,1,0,0,0,1,1,0
person_8,0,0,1,2,0,0,1,1,2,0
person_9,0,0,0,0,0,0,0,0,0,0


**Note:** element `(person_3,person_8)=2` because reponses of `person_3` and `person_8` are *influenced by both `factor_one` and `factor_two`*.
Now let's convert the above adjacency matrix to an edge, for it to be easily exported in CSV and imported eg. in `Gephi` or `networkx`

In [28]:
df_adj[df_adj >=1]

,person_0,person_1,person_2,person_3,person_4,person_5,person_6,person_7,person_8,person_9
person_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
person_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
person_2,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN
person_3,NaN,NaN,1.0,2.0,NaN,NaN,1.0,1.0,2.0,NaN
person_4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
person_5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
person_6,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN
person_7,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN
person_8,NaN,NaN,1.0,2.0,NaN,NaN,1.0,1.0,2.0,NaN
person_9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


get row and column indexes, with non zero values

In [36]:
import numpy as np
rows, cols = np.where(df_adj >= 1)

now get edge `weights` in a list, too 

In [38]:
weights = [df_adj.iloc[rows[i], cols[i]] for i in range(len(rows.tolist()))]

from this point and on we can create a list of edges as tuples

In [39]:
edge_list = zip(rows.tolist(), cols.tolist(), weights)

and create a `DataFrame` from it

In [40]:
edge_df = pd.DataFrame(edge_list, columns=['Src', 'Dest', 'Weight'])

We can of course export to CSV if we want

In [41]:
edge_df.to_csv('edges.csv', sep=',', index=None) # We leave index as None in order not to export a, rather useless, first field, to CSV 